## 라이브러리 호출

In [450]:
# pip install haversine
# !pip install geopandas

# pip install folium
# !jupyter nbextension install --sys-prefix --symlink --overwrite --py pydeck
# !jupyter nbextension enable --sys-prefix --py pydeck

In [449]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
# pd.options.display.float_format = '{:}'.format


import warnings
warnings.filterwarnings(action='ignore')

from haversine import haversine
import geopandas as gpd
from tqdm import tqdm

import folium
import json

import plotly.express as px

import mapboxgl

import pydeck as pdk

from mapboxgl.utils import df_to_geojson
import json

from mapboxgl.viz import ChoroplethViz
from mapboxgl.utils import create_color_stops

## 데이터 불러오기

In [36]:
df = pd.read_csv("서울시 전월세 데이터(전처리).csv")

In [4]:
df_police = pd.read_csv("경찰서 전처리 완료.csv", encoding = "euckr")

In [5]:
df_sub = pd.read_csv('수도권 지하철역 좌표 데이터.csv', encoding='euckr')

In [ ]:
df_cu = pd.read_csv('편의점.csv')

In [6]:
cctv = pd.read_csv("CCTV.csv")

## 전처리 -> V표시가 되어있는 셀만 실행

#### 위도, 경도 데이터 하나로 합치기 (V)

In [37]:
df_police["match"] = (df_police["lat"].astype(str) + "," + df_police["lng"].astype(str)) 

In [38]:
df["match"] = (df["lat"].astype(str) + "," + df["long"].astype(str)) 

In [39]:
cctv["match"] = (cctv["WGS84위도"].astype(str) + "," + cctv["WGS84경도"].astype(str)) 

#### 경찰서 위도, 경도를 기반으로 집과의 거리와 관공서명 생성 (V)

In [10]:
police_length = []
for i,j in tqdm(list(zip(df["item_id"], df["match"]))) :
    a, b = j.split(",")[0], j.split(",")[1]
    home = float(a), float(b)
    temp = []
    vs = []
    li = []
    for j, k in list(zip(df_police["치안센터명"],df_police["match"])) :
        a, b = k.split(",")[0], k.split(",")[1]
        police = float(a,), float(b)
        temp.append(haversine(home, police, unit = "km"))
        vs.append(haversine(home, police, unit = "km"))
        vs.append(j)
    li.append(min(temp))
    if  min(temp) == vs[vs.index(min(temp))] :
        li.append(vs[vs.index(min(temp))+1])
        li.append(i)
    police_length.append(li)

100%|████████████████████████████████████| 26165/26165 [00:31<00:00, 829.30it/s]


#### 집과 관공서의 거리를 데이터 프레임으로 전환 (V)

In [40]:
police_width  = pd.DataFrame(police_length, columns = ["경찰서_거리", "치안센터명", "item_id"])

In [ ]:
df.head(1)

In [41]:
df = pd.merge(df, police_width, on = "item_id")

#### 데이터프레임 컬럼 명 수정(거리) (V)

In [ ]:
df["거리"] = df["거리"].astype(str).str[:5].astype(float)

In [ ]:
df.rename({"거리" : "거리(km)"}, axis = 1, inplace = True)

#### CCTV 사용하지 않는 컬럼은 삭제하고 위도, 경도를 하나로 합친 파생변수 생성 및 경도 이상치 처리 (V)

In [43]:
cctv = cctv[cctv["WGS84경도"] < 130] # 경도에 260 데이터 삭제

In [45]:
cctv = cctv[(cctv["설치목적구분"] != "기타") & (cctv["설치목적구분"] != "교통정보수집")]

#### cctv 데이터 코드 알고리즘 구현 전 계획
- 설치목적구분, 위도경도, 설치대수 
- 특정 집의 위도 경도 기준으로 카메라의 위치를 비교하여 100m 내에 있는 카메라를 모두 수집

> 특정 집 모두 반복
 - 집을 대상으로 위치를 찾을 CCTV 전체 반복
     - 집과 CCTV간의 거리를 계산
     - 만약 100m 안에 있다면 리스트에 담기
     - 이때, 집의 아이디 정보, 해당 위도 경도에 설치 된 카메라 대수, 카메라의 위도 경도

#### CCTV 데이터 거리 계산 코드 구현

In [46]:
cctv_width = cctv[["구", "match", "설치목적구분", "카메라대수"]]

In [47]:
home_width = df[["local2", "match", "item_id"]]

##### 집이 어떤 특정 행정지역이라면 그에 맞는 CCTV가 설치 된 지역만 개수를 세라
- 집 위도 경도, 집 특정 구 반복
    - 만약 집의 특정 구가 관악구라면 CCTV 데이터도 관악구에 위치한 CCTV만 반복
        - CCTV를 반복 하면서 100m 내에 위치한 CCTV라면 리스트에 담기
    - 총 리스트에 담기(CCTV, 설치 대수, 집 정보)

In [48]:
gu = ['서대문구', '마포구', '성동구', '동대문구', '성북구', '중구', '종로구', '영등포구', '구로구',
       '양천구', '강서구', '은평구', '강남구', '서초구', '송파구', '노원구', '도봉구', '강북구',
       '용산구', '동작구', '중랑구', '금천구', '광진구', '강동구', '관악구']

In [49]:
cctv_stats = []
for i in tqdm(range(len(home_width))) : # range(len(home_width)) -> home_width.iloc[i]
    local, lat_lng, home_id = home_width.iloc[i]
    lat, lng = lat_lng.split(",")[0], lat_lng.split(",")[1]
    home = float(lat), float(lng)
    if local in gu :
        for cnt, k in list(zip(cctv_width["카메라대수"], cctv[cctv["구"] == local]["match"])) :
            temp = []
            latt, long = k.split(",")[0], k.split(",")[1]
            camera = float(latt), float(long)
            km = haversine(home, camera, unit = "km")
            if km <= 0.1 :
                temp.append(km)
                temp.append(cnt)
                temp.append(home_id)
            if temp != [] :
                cctv_stats.append(temp)

100%|█████████████████████████████████████| 26165/26165 [05:31<00:00, 78.89it/s]


In [ ]:
cctv_distance = pd.DataFrame(cctv_stats, columns = ["카메라_거리", "카메라대수","item_id"])

#### 지하철 데이터 취합

In [54]:
df_sub.loc[589]=['가락시장',37.492522,127.118234]

In [55]:
df_sub.columns=['지하철역','위도','경도']

In [56]:
df_sub["match"] = (df_sub["위도"].astype(str) + "," + df_sub["경도"].astype(str)) 

In [57]:
sub_length = []
for i,j in tqdm(list(zip(df["item_id"], df["match"]))) :
    a, b = j.split(",")[0], j.split(",")[1]
    home = float(a), float(b)
    temp = []
    vs = []
    li = []
    for j, k in list(zip(df_sub["지하철역"],df_sub["match"])) :
        a, b = k.split(",")[0], k.split(",")[1]
        police = float(a,), float(b)
        temp.append(haversine(home, police, unit = "km"))
        vs.append(haversine(home, police, unit = "km"))
        vs.append(j)
    li.append(min(temp))
    if  min(temp) == vs[vs.index(min(temp))] :
        li.append(vs[vs.index(min(temp))+1])
        li.append(i)
    sub_length.append(li)

100%|████████████████████████████████████| 26165/26165 [01:51<00:00, 234.31it/s]


In [59]:
sub_width = pd.DataFrame(sub_length, columns = ["지하철_거리", "지하철역", "item_id"])

In [60]:
df = pd.merge(df, sub_width, on = "item_id")

#### 편의점 데이터 취합

In [64]:
df_cu.head(1)

,MEGA_NM,MEGA_CD,CTY_CD,CTY_NM,TOTAL,위도,경도
0,서울특별시,11,11110,종로구,1,37.575244,126.968649


In [67]:
df_cu["match"] = (df_cu["위도"].astype(str) + "," + df_cu["경도"].astype(str)) 

store_length = []
for i,j in tqdm(list(zip(df["item_id"], df["match"]))) :
    a, b = j.split(",")[0], j.split(",")[1]
    home = float(a), float(b)
    temp = []
    vs = []
    li = []
    for k  in df_cu["match"] : 
        a, b = k.split(",")[0], k.split(",")[1]
        police = float(a,), float(b)
        temp.append(haversine(home, police, unit = "km"))
        vs.append(haversine(home, police, unit = "km"))
    li.append(min(temp))
    if  min(temp) == vs[vs.index(min(temp))] :
        li.append(i)
    store_length.append(li)

100%|█████████████████████████████████████| 26165/26165 [08:41<00:00, 50.22it/s]


In [68]:
store_width = pd.DataFrame(store_length, columns=['편의점_거리','item_id'])

In [70]:
df = pd.merge(df, store_width, on = "item_id")

#### CCTV 데이터 취합

In [437]:
df = pd.merge(df, cctv_distance, on = 'item_id')

#### 월세금액 이상치 치환

In [ ]:
df["월세금액"] = np.where(df["월세금액"] == df["월세금액"].max(), 95, df["월세금액"])

In [ ]:
df = df[df["전용면적_m2"] < 200] # 전용면적이 60평 아래인 집 데이터만 보존하기 위해 이상치 제거

#### km 형태의 거리 데이터를 소숫점 3자리 까지 표현

In [73]:
width_list = ['경찰서_거리', '지하철_거리', '편의점_거리']

In [75]:
for li in width_list :
    df[li] = df[li].astype(str).str[:5].astype(float)

#### 층별 데이터에서 0과 반지하, 옥탑방 데이터 수정

In [79]:
df["floor"].unique()

array(['1', '0', '5', '2', '3', '4', '반지하', '7', '9', '10', '옥탑방', '8',
       '6', '14', '11', '16', '12', '18', '15', '17', '19', '13', '21',
       '20', '22', '36', '25'], dtype=object)

In [83]:
df["floor"] = np.where(df["floor"] == "0", "1",df["floor"])

In [86]:
df['floorset']=np.where(df['floor'] == "반지하", 98,
np.where(df["floor"] =="옥탑방", 99, df['floor']))

df['floorset'] = df['floorset'].astype(int)
df["floorset"] = np.where((df['floorset'] <= 10),'10층 이하',
np.where(df['floorset'] <= 20,'20층 이하',
np.where(df['floorset'] <= 40,'40층 이하',df['floorset'])))

In [92]:
df["floorset"] = np.where(df["floorset"] == '98', "반지하",
np.where(df["floorset"] == '99', "옥탑방", df["floorset"]))

#### 날짜 데이터 전처리

In [221]:
lt = ["등기", "예정", "준공", "없음", "면적", "완료", "리모델링", "이전", "중순이","조율","신축중","말","승인","미정","신림","건축","미상","신축","중순"]

In [222]:
for i in lt : 
    for j in df[df["approve_date"].str.contains(i)].index :
        df.drop(j, inplace = True)

In [151]:
df["approve_date"] = df["approve_date"].str.replace("년","").str.replace("월","").str.replace("일","")

In [157]:
df["approve_date"]= df["approve_date"].str.replace(".","-").str.strip()

In [169]:
df["approve_date"] = df["approve_date"].str.replace(" ", "-")

In [228]:
df["month"] = df["month"].str.replace("-", "0").str.replace("/","0")

In [237]:
df.drop(df[df["month"] == "9"].index, inplace = True)

In [239]:
df.drop(df[df["month"] == "7"].index, inplace = True)

In [243]:
three_word = []
for i in df["month"].unique() :
    if len(i) == 3:
        three_word.append(i)

In [290]:
mapping = {"958" : '1995', "916" : '1991', "229" : '2022', "153" : '2015', "167" : '2016',
          "178" : '2017', "154" : '2015', "209" : '2020', "206" : '2020'}

In [299]:
for g in three_word :
    for i in df["month"].unique() :
        if g == i :
            df["month"] = df["month"].str.replace(g, mapping[g])

## 파일 저장

In [448]:
df.to_csv("전월세 및 거리계산 전처리.csv")

In [361]:
cctv_distance.to_csv("CCTV 거리 계산.csv")

## 시각화

In [306]:
# pip install folium

In [2]:
# us_cities = pd.read_csv(
#     "https://raw.githubusercontent.com/plotly/datasets/master/us-cities-top-1k.csv"
# )

# fig = px.scatter_mapbox(
#     df,
#     lat="lat",
#     lon="long",
#     hover_name="local2",
#     hover_data=["보증금액", "월세금액"],
#     color_discrete_sequence=["fuchsia"],
#     zoom=10,
#     height=500,
# )
# fig.update_layout(mapbox_style="open-street-map")
# fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
# fig.show()

In [437]:
df = pd.merge(df, cctv_distance, on = 'item_id')

In [399]:
geo_data = df_to_geojson(
    df=df,
    lat='lat',
    lon='long',
    # filename = "data/toilet_seoul.geojson"
)

type(geo_data)

geojson.feature.FeatureCollection

In [408]:
gdf = gpd.GeoDataFrame(df)

geopandas.geodataframe.GeoDataFrame

In [3]:
#                                                               ㅇㅇㅇㅋ# 중심 경도 위도 (ex. 남산타워 좌표값)
# lon, lat = 126.987867837993, 37.5511225714939
# center = [lon, lat]

# # 색상 범주 설정
# color_breaks = list(range(1000, 12000, 2000))
# color_stops = create_color_stops(color_breaks, colors='BuPu')

# # 2D 지도 생성
# viz = ChoroplethViz(
#     access_token = mapbox_key,
#     data = geo_data,
#     color_property = '전용평당가',
#     color_stops = color_stops,
#     center = center,
#     zoom = 10
# )

# # 출력
# viz.show()

In [441]:
df2=df[['보증금액','월세금액','local2','lat','long','경찰서_거리','치안센터명', '지하철_거리', '지하철역', '편의점_거리','카메라_거리', '카메라대수']]
df2=df2[df2['lat']!='']
df2=df2[df2['long']!='']
df2['lat']=df2['lat'].astype(float)
df2['lon']=df2['long'].astype(float)

geo_data = df_to_geojson(
    df=df2,
    properties=['보증금액','월세금액','local2','경찰서_거리','치안센터명', '지하철_거리', '지하철역', '편의점_거리','카메라_거리', '카메라대수'],
    lat='lat', lon='lon',
    precision=11,
    filename = "home.geojson"
)

geo_data_2 = 'home.geojson'

In [396]:
mapbox_key = "pk.eyJ1IjoianVod2FuMiIsImEiOiJjbGF2amRkejMwNHd1M3ZsbjM0MmlneXdxIn0.KzYlIn8tpJcw2zQH4rEqtA"

In [1]:
# with open(geo_data_2) as f:
#     data = json.loads(f.read())
    
# from mapboxgl.viz import *

# token = mapbox_key
# center = [126.933400, 37.501156]

# viz = CircleViz(
#     data,
#     access_token=token,
#     center=center,
#     color_default = 'lightBlue',
#     zoom=10)

# viz.show()
# viz.create_html('home.html')

# from mapboxgl.utils import (
#     create_numeric_stops,
#     create_color_stops
# )

# color_stops = create_color_stops([1,10,50,100], colors='RdYlGn')

# cviz = ClusteredCircleViz(data,
#                          access_token=token,
#                          color_stops=color_stops,
#                          radius_stops=[[1,5], [10, 10], [50, 15], [100, 20]],
#                          radius_default=2,
#                          cluster_maxzoom=10,

#                          cluster_radius=30,
#                          label_size=12,
#                          opacity=0.9,
#                          center=center,
#                          zoom=10)

# cviz.show()
# cviz.create_html('home_cluster.html')